<b>Local Magnitude Estimator</b><br>
This Notebook is designed to demonstrate how to download a set of waveforms from stations near a catalog earthquake and use them to estimate the local magnitude.


In [1]:
import os
from math import log10
from obspy import UTCDateTime, read, read_inventory
from obspy.geodetics import gps2dist_azimuth
from obspy.clients.fdsn.mass_downloader import CircularDomain, Restrictions, MassDownloader
from obspy.clients.fdsn import Client
client = Client("IRIS")

The following code will set the variables for the earthquake catalog search.  If a catalog event is not yet available, the values needed for future code are the origin time and the latitude and longitude.<br><br>
In the predefined example, the values are set for a small earthquake in the Eastern Tennessee Seismic Zone.  The USGS page for this event can be found here:<br> <a href=https://earthquake.usgs.gov/earthquakes/eventpage/se60300191>https://earthquake.usgs.gov/earthquakes/eventpage/se60300191</a>

In [5]:
catstartt = UTCDateTime("2020-01-01")
catendt = UTCDateTime("2020-08-15")
lon = -84
lat = 36.43
rad = 1 # 111.32 km
cat = client.get_events(starttime=catstartt, endtime=catendt, latitude=lat, longitude=lon, maxradius=rad, minmagnitude=3)

Printing the catalog will show the events which have been returned by the search.  Note what the catalog magnitude value is and what type it is.  The origin time is stored in a variable.

In [6]:
print(cat)

1 Event(s) in Catalog:
2020-01-20T19:12:11.260000Z | +36.431,  -84.027 | 3.8  Mwr


Next we set the variables for the waveform search.

In [8]:
origint = UTCDateTime(cat[0].origins[0].time.datetime)
wfstartt = origint - 10
wfendt = origint + 50
stamaxrad = 1 # 111.32 km
domain = CircularDomain(latitude=lat, longitude=lon, minradius=0, maxradius=stamaxrad)
restrictions = Restrictions(starttime=wfstartt, endtime=wfendt, channel_priorities=["HH[NE12]", "BH[NE12]"], location_priorities=["", "00", "10"])

The following code performs the waveform search.  It looks for stations from many different FDSN clients, so it can take several minutes to complete.  It will produce a lot of information in pink that describes the different steps through the search process.

In [9]:
mdl = MassDownloader()
mdl.download(domain,restrictions, mseed_storage="waveforms", stationxml_storage="stations")

[2024-08-28 07:58:56,634] - obspy.clients.fdsn.mass_downloader - INFO: Initializing FDSN client(s) for AUSPASS, BGR, EIDA, EMSC, ETH, GEOFON, GEONET, GFZ, ICGC, IESDMC, INGV, IPGP, ISC, KNMI, KOERI, LMU, NCEDC, NIEP, NOA, RESIF, RESIFPH5, SCEDC, TEXNET, UIB-NORSAR, USGS, USP, ORFEUS, IRIS.
[2024-08-28 07:58:56,887] - obspy.clients.fdsn.mass_downloader - WARNING: Failed to initialize client 'TEXNET'.
[2024-08-28 07:58:57,388] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'USGS' as it does not have 'dataselect' and/or 'station' services.
[2024-08-28 07:58:57,696] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'ISC' as it does not have 'dataselect' and/or 'station' services.
[2024-08-28 07:58:57,849] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'EMSC' as it does not have 'dataselect' and/or 'station' services.
[2024-08-28 07:58:58,586] - obspy.clients.fdsn.mass_downloader - INFO: Cannot use client 'RESIFPH5' as it does not have 'dataselec

{'AUSPASS': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x10a8d25d0>,
 'BGR': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x10a699f50>,
 'EIDA': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x10a8f4b90>,
 'ETH': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x10a7dbf10>,
 'GEOFON': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x10a98d050>,
 'GEONET': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x1066efb90>,
 'GFZ': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x108ef7750>,
 'ICGC': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x10a87a550>,
 'IESDMC': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x107fbeb90>,
 'INGV': <obspy.clients.fdsn.mass_downloader.download_helpers.ClientDownloadHelper at 0x10a732

This code loops through each of the downloaded waveforms.  It removes the instrument response using a defined pre-filter and converts to displacement.  The maximum displacement is determined, along with the distance from the event to the station.  The local magnitude is calculated using the approximate emprical formula from Bullen and Bolt (1985).  The average magnitude for each station-channel is reported at the end.

In [10]:
filt = ( 0.5, 1, 40, 50 )
ms = 0.0
n = 0
for file in os.listdir("waveforms"):
    wavfile = "waveforms/" + file
    stafile = "stations/" + file.split(".")[0] + "." + file.split(".")[1] + ".xml"
    print (wavfile, stafile)
    st = read(wavfile)
    tr = st[0]
    inv = read_inventory(stafile)
    tr.remove_response(inventory=inv, pre_filt=filt, output="DISP")
    disp = abs( tr.max() * 1e6 ) # displacement in microns
    stalat = inv.networks[0].stations[0].latitude
    stalon = inv.networks[0].stations[0].longitude
    evelat = cat[0].origins[0].latitude
    evelon = cat[0].origins[0].longitude
    dist = gps2dist_azimuth(stalat, stalon, evelat, evelon )[0] / 1000 # distance in kilometers
    magl = log10(disp) + 2.56 * log10(dist) - 1.67
    print("displacement =",disp,", distance =",dist,", local mag =",magl)
    ms += magl
    n += 1
print ()

waveforms/N4.T50A.00.HH1__20200120T191201Z__20200120T191301Z.mseed stations/N4.T50A.xml
displacement = 5.2845523235707486 , distance = 97.6787740409144 , local mag = 4.1468967147509606
waveforms/US.TZTN.00.BH2__20200120T191201Z__20200120T191301Z.mseed stations/US.TZTN.xml
displacement = 13.017583246232892 , distance = 44.62745650562326 , local mag = 3.667511830238496
waveforms/IM.TKL..BHE__20200120T191201Z__20200120T191301Z.mseed stations/IM.TKL.xml
displacement = 13.60724088833879 , distance = 88.69248944267055 , local mag = 4.45036039623072
waveforms/N4.T50A.00.HH2__20200120T191201Z__20200120T191301Z.mseed stations/N4.T50A.xml
displacement = 4.728445859199891 , distance = 97.6787740409144 , local mag = 4.098606933163199
waveforms/IM.TKL..BHN__20200120T191201Z__20200120T191301Z.mseed stations/IM.TKL.xml
displacement = 5.069890540653064 , distance = 88.69248944267055 , local mag = 4.021588906062301
waveforms/US.TZTN.00.BH1__20200120T191201Z__20200120T191301Z.mseed stations/US.TZTN.xml


In [11]:
am = ms / n

print("local magnitude: " + str(am))

local magnitude: 4.033903255856484
